In [1]:
import pandas as pd
import re
import textacy

In [2]:
fk_df = pd.read_csv('ForestKeywordsProcessed_v1.csv',index_col=0)

In [3]:
fk_df.head(10)

,genus,species,common_name,range_state
0,Abarema,jupunba,Ingarana;Tento Azul;Huruasa,GY
1,Acacia,auriculiformis,Black Wattle;Brown Salwood;Mangge Hutan;Tongke...,AU;ID;MY;PG;TH;GB;US
2,Acacia,mangium,Black Wattle;Brown Salwood;Mangge Hutan;Tongke...,AU;ID;MY;PG;TH;GB;US
3,Adina,cordifolia,Khvao;Kwao;Haldu;Lasi;Thom;Meraga;Hnaw;Adina;H...,KH;IN;ID;LA;MY;MM;PH;LK;TH;VN
4,Adina,microcephala,Matumi;Rhodesian Redwood;Mugonha,DZ;ES;ES;EG;LY;PT;ES;MA;SD;TN;EH;BI;KM;DJ;ER;E...
5,Afrormosia,elata,Obang;Obang;Assamela;Kokrodua;Ole;Bohala;Mohol...,CM;CF;CI;GH;CD;FR
6,Afzelia,africana,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US
7,Afzelia,bella,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US
8,Afzelia,bipindensis,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US
9,Afzelia,cuanzensis,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US


In [4]:
base_df = pd.read_csv('combined_species_3.csv',index_col=0)
base_df.head(10)

,family,genus,species,sub_sp,common_name,range_state,is_tropical,is_coniferous,flag
0,NaN,Abarema,abbottii,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,Abarema,alexandri,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,Abarema,bigemina,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,Abarema,callejasii,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,Abarema,centiflora,NaN,NaN,NaN,NaN,NaN,1.0
5,NaN,Abarema,cochleata,NaN,NaN,NaN,NaN,NaN,1.0
6,NaN,Abarema,cochliocarpos,NaN,NaN,NaN,NaN,NaN,0.0
7,NaN,Abarema,commutata,NaN,NaN,NaN,NaN,NaN,0.0
8,NaN,Abarema,curvicarpa,NaN,orelha de negro,NaN,NaN,NaN,0.0
9,NaN,Abarema,filamentosa,NaN,NaN,NaN,NaN,NaN,1.0


In [5]:
#Merge 

for i, row in fk_df.iterrows():
    gn = row['genus']
    sp = row['species']
    
    # find in base_df
    idx = base_df.loc[( base_df['genus'] == gn ) & (base_df['species'] == sp)].index.tolist()
    if len(idx) == 0 :
        
        cn = row['common_name']
        if type(cn) == str :
            cn = cn.lower()
        else:
            cn = None
        #insert into base_df
        tmp_dict = {
            'family'  : None,
            'genus' : gn ,
            'species' : sp,
            'sub_sp' : None,
            'common_name' : cn,
            'range_state': row['range_state'],
            'is_tropical' : 1,
            'is_coniferous': None,
            'flag' : 0,
        }
        base_df = base_df.append(tmp_dict,ignore_index=True)
    else :
        idx = idx[0]
        # merge
        cn0 = base_df.loc[idx,'common_name']
        rs0 = base_df.loc[idx,'range_state']
        
        cn1  = row['common_name']
        rs1  = row['range_state']
        
        if type(cn0) != str :
            cn0 = []
        else:
            cn0 = cn0.lower()
            cn0 = cn0.split(';')
            
        if type(cn1) != str :
            cn1 = []
        else :
            cn1 = cn1.lower()
            cn1 = cn1.split(';')
        
        cn0.extend(cn1)
        cn = list(set(cn0))
        cn = ';'.join(cn)
        
        if type(rs0) != str :
            rs0 = []
        else:
            rs0 = rs0.split(';')
            
        if type(rs1) != str :
            rs1 = []
        else :
            rs1 = rs1.split(';')
        
        rs0.extend(rs1)
        rs = list(set(rs0))
        rs = ';'.join(rs)
        
        base_df.loc[idx,'is_tropical'] = 1
        base_df.loc[idx,'common_name'] = cn
        base_df.loc[idx,'range_state'] = rs
        

In [6]:
base_df = base_df.sort_values(by=['genus','species'])

In [7]:
base_df.to_csv('combined_species_4.csv')



In [8]:
hr_df = pd.read_csv('WWF_HighRisk_final.csv',index_col=None)
del hr_df['Unnamed: 0']
hr_df.head(10)

,genus,species,common_name,origin,conduit
0,Shorea,NaN,meranti;lauan;lawaan;seraya;balau;bangkirai;ph...,LA;MM;VN,CN
1,Shorea,platyclados,marsolok,ID,NaN
2,Dalbergia,NaN,rio;bahia;brazilian rosewood;sheesham;palisand...,LA;MM;PA;VN;BR,CN
3,Pinus,koraiensis,korean pine;korean cedar,RU,CN
4,Quercus,mongolica,oak,RU,CN
5,Pterocarpus,NaN,padauk;padouk;mukwa;narra;amboyna,LA;MM;VN,NaN
6,Pericopsis,NaN,african teak;afromosia;afrormosia;satinwood;ye...,NaN,CN
7,Dipterocarpus,NaN,keruing;asian mahogany;gurjan.,ID;LA;MM;VN,NaN
8,Swietenia,NaN,mahogany,PA;PE,CN
9,Swietenia,macrophylla,marsolok,PA,NaN


In [9]:
for i, row in hr_df.iterrows():
    gn = row['genus']
    sp = row['species']
    
    # find in base_df
    idx = base_df.loc[( base_df['genus'] == gn ) & (base_df['species'] == sp)].index.tolist()
    if len(idx) == 0 :
        
        cn = row['common_name']
        if type(cn) == str :
            cn = cn.lower()
        else:
            cn = None
        #insert into base_df
        tmp_dict = {
            'family'  : None,
            'genus' : gn ,
            'species' : sp,
            'sub_sp' : None,
            'common_name' : cn,
            'range_state': row['origin'],
            'is_tropical' : 1,
            'is_coniferous': None,
            'flag' : 0,
        }
        base_df = base_df.append(tmp_dict,ignore_index=True)
    else :
        idx = idx[0]
        # merge
        cn0 = base_df.loc[idx,'common_name']
        rs0 = base_df.loc[idx,'range_state']
        
        cn1  = row['common_name']
        rs1  = row['origin']
        
        if type(cn0) != str :
            cn0 = []
        else:
            cn0 = cn0.lower()
            cn0 = cn0.split(';')
            
        if type(cn1) != str :
            cn1 = []
        else :
            cn1 = cn1.lower()
            cn1 = cn1.split(';')
        
        cn0.extend(cn1)
        cn = list(set(cn0))
        cn = ';'.join(cn)
        
        if type(rs0) != str :
            rs0 = []
        else:
            rs0 = rs0.split(';')
            
        if type(rs1) != str :
            rs1 = []
        else :
            rs1 = rs1.split(';')
        
        rs0.extend(rs1)
        rs = list(set(rs0))
        rs = ';'.join(rs)
        
        base_df.loc[idx,'common_name'] = cn
        base_df.loc[idx,'range_state'] = rs

In [10]:
base_df.head(10)

,family,genus,species,sub_sp,common_name,range_state,is_tropical,is_coniferous,flag
0,NaN,Abarema,abbottii,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,Abarema,alexandri,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,Abarema,bigemina,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,Abarema,callejasii,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,Abarema,centiflora,NaN,NaN,NaN,NaN,NaN,1.0
5,NaN,Abarema,cochleata,NaN,NaN,NaN,NaN,NaN,1.0
6,NaN,Abarema,cochliocarpos,NaN,NaN,NaN,NaN,NaN,0.0
7,NaN,Abarema,commutata,NaN,NaN,NaN,NaN,NaN,0.0
8,NaN,Abarema,curvicarpa,NaN,orelha de negro,NaN,NaN,NaN,0.0
9,NaN,Abarema,filamentosa,NaN,NaN,NaN,NaN,NaN,1.0


In [12]:
base_df = base_df.sort_values(by=['genus','species'])
base_df.to_csv('combined_species_5.csv')

In [13]:
base_df.head(10)

,family,genus,species,sub_sp,common_name,range_state,is_tropical,is_coniferous,flag
0,NaN,Abarema,abbottii,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,Abarema,alexandri,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,Abarema,bigemina,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,Abarema,callejasii,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,Abarema,centiflora,NaN,NaN,NaN,NaN,NaN,1.0
5,NaN,Abarema,cochleata,NaN,NaN,NaN,NaN,NaN,1.0
6,NaN,Abarema,cochliocarpos,NaN,NaN,NaN,NaN,NaN,0.0
7,NaN,Abarema,commutata,NaN,NaN,NaN,NaN,NaN,0.0
8,NaN,Abarema,curvicarpa,NaN,orelha de negro,NaN,NaN,NaN,0.0
9,NaN,Abarema,filamentosa,NaN,NaN,NaN,NaN,NaN,1.0
